In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
import matplotlib as plt

In [86]:
## Import Data from Local  - Churn_Modelling.csv file
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [82]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [83]:
## Copying the master DF into df1

data = df

In [87]:
## Drop unwanted column - Feature Scalling
data.drop(['Surname','RowNumber','CustomerId'],axis=1,inplace=True)
data.head(3)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [88]:
## Geography and Gender are 2 categorial variables
## We can apply some Encoding practise known as LabelEncoder 

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [89]:
## To check how many values are there in an individual Columns

data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [90]:
## One Hot Encoding is used in those scenario where we have more than 2 values in a features
## WHY: If France is 0; Spain is 1; Germany is 2. 
## Then it will consider as Germany is greater than Spain or France. 
## Which we dont want. Hence One hot encoding will place 0 in each value except its own
## France = 0; Spain = 0; Germany = 1.
## France = 0; Spain = 1; Germany = 0.
## France = 1; Spain = 0; Germany = 0.

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [92]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [ ]:
geo_encoded_df## Here we are concatenating the value of sparse matrix + names of One hot encodding for Geography column.

 = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [94]:
## Now the time came, We need to concatenate the above variable of One Hot encoding with existing dataFrame
## geo_encoded_df + existing DataFrame (which is data)
## Step 1 - Frist drop the Original column  Geography and same time do concatenate

data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [96]:
## For future purpose and using it in Deployment or in streamlit app. 
## We have to save it in form of pickle file

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

Divide the Dataset into Train - Test Split

In [100]:
X = data.drop(['Exited'],axis=1)
y = data.Exited

## split the data in training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Scaliing the values of features of X_train and X_test
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)



In [101]:
X_train

array([[ 1.67588802, -1.10022343, -0.94183739, ..., -1.00388816,
         1.74875572, -0.57987798],
       [ 0.30320673,  0.9089063 , -0.17987213, ..., -1.00388816,
         1.74875572, -0.57987798],
       [-0.10235819,  0.9089063 ,  0.96307575, ..., -1.00388816,
         1.74875572, -0.57987798],
       ...,
       [ 0.86475817, -1.10022343, -0.08462648, ...,  0.9961269 ,
        -0.57183516, -0.57987798],
       [ 0.15761932,  0.9089063 ,  0.39160181, ...,  0.9961269 ,
        -0.57183516, -0.57987798],
       [ 0.46959234,  0.9089063 ,  1.15356706, ..., -1.00388816,
         1.74875572, -0.57987798]])

In [102]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [104]:
## importing library for tensorflow for ANN Implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [106]:
(X_train.shape[1],)

(12,)

In [107]:
## Start building ANN model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # 1st Hidden layer connected with Input layer. input_shape is written for 1st layer only
    Dense(32,activation='relu'), # Hidden layer 2
    Dense(1,activation='sigmoid') ## Output layer
])

e:\DataScience\DataScience\DeepLearning\ANN\ANN_Practise_New\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [108]:
## To see how many parameters are there in the above Sequesntial model where 2 Hidden layer (64 and 32) n 1 output layer
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [111]:
## Another way of defining OPTIMIZERS
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [112]:
## For doing Forward and Backward Propogation, We have to compile it first

model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [126]:
## When we train the model, we can find the logs in these particular directory
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [127]:
## Setting up Early Stopping. 
## It is required bcoz when my model is executing and i can see the accuracy is not OR hardly getting changed after 
## particular epocs. By that time I dont wnat to run rest of the epocs.
## EG - : Suppose I specify 100 epocs but at 10 epocs I see that my accuracy is hardly chaging then I will early stopped
##          it. Hence there will be a limited resource used


early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [128]:
## Now Final Stage: ------ Training the MODEL

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callbacks]
)

Epoch 1/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8593 - loss: 0.3388 - val_accuracy: 0.8606 - val_loss: 0.3443
Epoch 2/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8600 - loss: 0.3346 - val_accuracy: 0.8624 - val_loss: 0.3441
Epoch 3/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8631 - loss: 0.3299 - val_accuracy: 0.8636 - val_loss: 0.3459
Epoch 4/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8589 - loss: 0.3351 - val_accuracy: 0.8664 - val_loss: 0.3401
Epoch 5/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8628 - loss: 0.3208 - val_accuracy: 0.8648 - val_loss: 0.3418
Epoch 6/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8687 - loss: 0.3193 - val_accuracy: 0.8606 - val_loss: 0.3542
Epoch 7/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8711 - loss: 0.3148 - val_accuracy: 0.8588 - val_loss: 0.3455
Epoch 8/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8654 - loss: 0.3229 - val_accu

In [129]:
## Saving the model to h5 file format

model.save('model.h5')

In [120]:
## Load TensorBoard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [124]:
%reload_ext tensorboard

In [125]:
%tensorboard --logdir logs/fit20250513-195116

Reusing TensorBoard on port 6007 (pid 68000), started 0:00:14 ago. (Use '!kill 68000' to kill it.)